## 3.3 分组运算

分组运算（Groupby）是指按某一特征（字段）将一个大数据表分成几张小表，分表后经过统计处理再将结果重组。

In [116]:
 
import pandas as pd
import numpy as np

### 3.3.1  分组groupby

分组既可以对Series分组，也可以对DataFrame分组，支持使用一个特征及多个特征作为分组条件。执行 groupby 命令后数据并未被真正拆分，只是在访问组中的数据时才会执行拆分操作，这会使得分组操作变得快速且节约存储空间。
使用get_group方法可获取某一组中的所有记录。

In [139]:
import statsmodels.api as sm
data = sm.datasets.anes96.load_pandas().data
print(data.head(10))  # 显示数据前5行

    popul  TVnews  selfLR  ClinLR  DoleLR  PID   age  educ  income  vote  \
0     0.0     7.0     7.0     1.0     6.0  6.0  36.0   3.0     1.0   1.0   
1   190.0     1.0     3.0     3.0     5.0  1.0  20.0   4.0     1.0   0.0   
2    31.0     7.0     2.0     2.0     6.0  1.0  24.0   6.0     1.0   0.0   
3    83.0     4.0     3.0     4.0     5.0  1.0  28.0   6.0     1.0   0.0   
4   640.0     7.0     5.0     6.0     4.0  0.0  68.0   6.0     1.0   0.0   
5   110.0     3.0     3.0     4.0     6.0  1.0  21.0   4.0     1.0   0.0   
6   100.0     7.0     5.0     6.0     4.0  1.0  77.0   4.0     1.0   0.0   
7    31.0     1.0     5.0     4.0     5.0  4.0  21.0   4.0     1.0   0.0   
8   180.0     7.0     4.0     6.0     3.0  3.0  31.0   4.0     1.0   0.0   
9  2800.0     0.0     3.0     3.0     7.0  0.0  39.0   3.0     1.0   0.0   

   logpopul  
0 -2.302585  
1  5.247550  
2  3.437208  
3  4.420045  
4  6.461624  
5  4.701389  
6  4.606170  
7  3.437208  
8  5.193512  
9  7.937410  


In [140]:
grp = data.groupby('educ') # 按单特征分组
print(len(grp))
grp = data.groupby(['educ','vote']) # 按两特征分组
print(len(grp))
grp = data.groupby(lambda n: n%2) # 按索引值奇偶分组
print(len(grp))
print(grp.get_group(0).head())

grp = data.groupby(['vote'])
print(grp['vote'].count()) # 求每组人数

df = grp['age'].mean().reset_index()
print(type(df))
print(df)

7
14
2
   popul  TVnews  selfLR  ClinLR  DoleLR  PID   age  educ  income  vote  \
0    0.0     7.0     7.0     1.0     6.0  6.0  36.0   3.0     1.0   1.0   
2   31.0     7.0     2.0     2.0     6.0  1.0  24.0   6.0     1.0   0.0   
4  640.0     7.0     5.0     6.0     4.0  0.0  68.0   6.0     1.0   0.0   
6  100.0     7.0     5.0     6.0     4.0  1.0  77.0   4.0     1.0   0.0   
8  180.0     7.0     4.0     6.0     3.0  3.0  31.0   4.0     1.0   0.0   

   logpopul  
0 -2.302585  
2  3.437208  
4  6.461624  
6  4.606170  
8  5.193512  
vote
0.0    551
1.0    393
Name: vote, dtype: int64
<class 'pandas.core.frame.DataFrame'>
   vote        age
0   0.0  46.299456
1   1.0  48.086514


### 3.3.2  聚合agg


聚合（Agg）可以对每组中的数据进行聚合运算，即把多个值按指定方式转换成一个值。Agg 的参数是处理函数，它将列中的数据转给处理函数

In [119]:
data = sm.datasets.ccard.load_pandas().data
print(data.head()) # 显示数据前5行
print('*******************************')
grp = data.groupby('OWNRENT')
print(grp.agg(np.mean)) # 调用聚合函数
print('*******************************')
print(grp.agg(lambda df: df.loc[(df.INCOME.idxmax())]))

   AVGEXP   AGE  INCOME  INCOMESQ  OWNRENT
0  124.98  38.0    4.52   20.4304      1.0
1    9.85  33.0    2.42    5.8564      0.0
2   15.00  34.0    4.50   20.2500      1.0
3  137.87  31.0    2.54    6.4516      0.0
4  546.50  32.0    9.79   95.8441      1.0
*******************************
             AVGEXP        AGE    INCOME   INCOMESQ
OWNRENT                                            
0.0      203.000667  28.866667  2.818667   8.764329
1.0      361.751111  35.296296  4.467778  24.490293
*******************************
         AVGEXP   AGE  INCOME  INCOMESQ
OWNRENT                                
0.0      306.03  41.0     6.0      36.0
1.0      548.03  40.0    10.0     100.0


### 3.3.3  转换Transform

转换（Transform）是将数据表中的每个元素按不同组进行不同的转换处理，转换之后的行索引和列索引不变，只有内容改变。

In [120]:
data = sm.datasets.ccard.load_pandas().data # 读取数据
grp = data.groupby('OWNRENT')
data['NEW_INCOME'] = grp['INCOME'].transform(lambda x: x - x.mean()) # 按组转换，将收入（INCOME）的实际值转换为INCOME减去该组的均值，
print(data[['INCOME', 'NEW_INCOME', 'OWNRENT']].head())

   INCOME  NEW_INCOME  OWNRENT
0    4.52    0.052222      1.0
1    2.42   -0.398667      0.0
2    4.50    0.032222      1.0
3    2.54   -0.278667      0.0
4    9.79    5.322222      1.0


### 3.3.4 应用Apply

相对于前几种方法，应用（Apply）更加灵活，并且能实现前几种处理的功能。其处理函数的输入是各组的 DataFrame，返回值可以是数值、Series，DataFrame。

In [121]:
print(grp.apply(np.mean))
print('*******************************')
print(grp['INCOME'].apply(lambda x: x - x.mean()).head()) # 同transform
print('*******************************')
print(grp.apply(lambda df: df if df['INCOME'].mean() < 3 else None).head())
print('*******************************')
print(grp.apply(lambda df: df.head(3) if df['INCOME'].mean() < 3 else None).head())

             AVGEXP        AGE    INCOME   INCOMESQ  OWNRENT    NEW_INCOME
OWNRENT                                                                   
0.0      203.000667  28.866667  2.818667   8.764329      0.0 -4.539579e-16
1.0      361.751111  35.296296  4.467778  24.490293      1.0  5.921189e-16
*******************************
0    0.052222
1   -0.398667
2    0.032222
3   -0.278667
4    5.322222
Name: INCOME, dtype: float64
*******************************
           AVGEXP   AGE  INCOME  INCOMESQ  OWNRENT  NEW_INCOME
OWNRENT                                                       
0.0     1    9.85  33.0    2.42    5.8564      0.0   -0.398667
        3  137.87  31.0    2.54    6.4516      0.0   -0.278667
        5   92.00  23.0    2.50    6.2500      0.0   -0.318667
        6   40.83  28.0    3.96   15.6816      0.0    1.141333
        9   52.58  28.0    3.20   10.2400      0.0    0.381333
*******************************
           AVGEXP   AGE  INCOME  INCOMESQ  OWNRENT  NEW_INCOME
O

## 3.4日期时间处理

Python的标准库提供了datetime系列工具，，datetime可同时处理时间信息和日期信息。

In [141]:
# 时间点
from datetime import datetime
d1 = datetime.now() # 获取当前时间
print(d1)
print(d1.year, d1.month, d1.day, d1.hour, d1.minute, d1.second)
d2 = datetime(2019, 3, 27) # 通过指定日期构造datetime
print(d2)



2022-03-01 16:51:57.925985
2022 3 1 16 51 57
2019-03-27 00:00:00


时间段 timedelta 用于表示两个时间点的差值，可以通过 datetime 数据相减得到，也可以通过指定具体时间差的方式构造。

In [123]:
# 时间段
from datetime import timedelta
delta = d1-d2 # 通过时间日期相减获取
print(type(delta))
print(delta)
delta = timedelta(days=3) # 通过指定时定差获取
print(d1+delta)# 利用时间段计算新日期时间

<class 'datetime.timedelta'>
1070 days, 15:32:50.925561
2022-03-04 15:32:50.925561


时间戳是指格林尼治时间自1970年1月1日零时至当前时间的总秒数，使用时间戳的好处在于节省存储空间，且不受不同系统之间的日期时间格式限制。

可以使用time.time函数获取当前时间戳，也可以使用time.mktime函数将datetime类型数据转换成时间戳，或者从字符串格式转换。

In [147]:
# 时间戳
import time
print(time.time())

d = datetime.now()
t = time.mktime(d.timetuple()) # 从datetime格式转换
print(t)
print(time.mktime(time.strptime("2019-03-27", "%Y-%m-%d"))) # 从字符串转换
print('*******************************')
print(datetime.fromtimestamp(t)) 
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(t)))

1646124809.8709924
1646124809.0
1553616000.0
*******************************
2022-03-01 16:53:29
2022-03-01 16:53:29


字符串类型转换成时间类型可使用datetime自带的strptime函数，在使用时需要指定时间格式。
在事先不确定时间日期格式的情况下，可以使用dateutil.parser中的parse方法自动识别字符串的时间类型，使用使用datetime的strftime方法可以指定格式。

In [125]:
# 时间类型转换
d = datetime.strptime('2019-03-27', '%Y-%m-%d')
print(d)

from dateutil.parser import parse
d = parse('2019/03/27')
print(d)
print(str(d))

print(d.strftime("%Y/%m/%d %H:%M:%S"))

2019-03-27 00:00:00
2019-03-27 00:00:00
2019-03-27 00:00:00
2019/03/27 00:00:00


### 3.4.2  Pandas日期时间处理

Pandas支持时间点Timestamp、时间间隔Timedelta和时间段Period三种时间类型。

1.时间点
Pandas 最基本的数据类型为时间点，它继承自 datetime，可使用 to_datetime 方法从字符串格式或者datetime格式转换。

2.时间间隔
Pandas中的时间间隔类似于datetime工具中的时间段，可以通过两个时间点相减获得。它的属性days，seconds可以查看具体的天数以及一天以内的秒数。时间间隔可以通过pd.Timedelta函数创建，并利用它与时间点的计算构造新的时间点。

3.时间段
时间段描述的也是时间区间，但与时间间隔不同的是，它包含起始时间和终止时间。

In [126]:
# 时间点TimeStamp

t = pd.to_datetime('2019-03-01 00:00:00') # 从字符串转换
print(type(t), t)
t = pd.to_datetime(datetime.now()) # 从datetime格式转换
print(type(t), t)

# 时间间隔
t1 = pd.to_datetime('2019-03-01 00:00:00')
t2 = pd.to_datetime(datetime.now())
delta = t2-t1 # 通过TimeStamp相减获取
print(type(delta), delta, delta.days, delta.seconds)

delta = pd.Timedelta(days=27) # 构造时间间隔为27天
print(t2 + delta)



<class 'pandas._libs.tslibs.timestamps.Timestamp'> 2019-03-01 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'> 2022-03-01 15:32:50.990577
<class 'pandas._libs.tslibs.timedeltas.Timedelta'> 1096 days 15:32:50.990577 1096 55970
2022-03-28 15:32:50.990577


In [127]:
# 时间段Period
t = pd.to_datetime(datetime.now())
p = pd.Period(t, freq='H')
print(p, p.start_time, p.end_time) # 显示时间段起止时间

# 批量转换
arr = ['2019-03-01','2019-03-02','2019-03-03']
df = pd.DataFrame({'d':arr})
df['d'] = pd.to_datetime(df['d'])
print(df)

2022-03-01 15:00 2022-03-01 15:00:00 2022-03-01 15:59:59.999999999
           d
0 2019-03-01
1 2019-03-02
2 2019-03-03


### 3.4.3  时间序列操作

时间序列是一系列相同格式的数据按时间排列后组成的数列。在时间序列数据的清洗和准备时，会用到大量的与Pandas日期相关的操作，如按时间筛选、切分、统计、聚合、采样、去重、偏移等。


In [128]:
# 时间日期类型索引
df.index = pd.to_datetime(df['d']) # 本例中使用了上例中构造的df[‘d’]
print(df.index)
print('*******************************')
df = pd.DataFrame()
df['date'] = pd.date_range(start='2017-12-30',end='2019-01-05',freq='d') # 创建时间数据
df['val'] = df['date'].apply(lambda x: x.weekday()) # 计算该日是星期几
df.set_index('date', inplace = True) # 设置时间索引
print(df.head()) # 显示前三条



DatetimeIndex(['2019-03-01', '2019-03-02', '2019-03-03'], dtype='datetime64[ns]', name='d', freq=None)
*******************************
            val
date           
2017-12-30    5
2017-12-31    6
2018-01-01    0
2018-01-02    1
2018-01-03    2



使用to_period方法和to_timestamp方法可以在时间段类型索引和时间点索引之间相互转换。

In [129]:
# 时间段类型索引
df_period = df.to_period(freq='M') # 按月创建时间段
print(type(df_period.index)) # 查看类型
print(len(df_period)) # 查看记录个数，与原记录个数一致
print(df_period.head(5))
print('*******************************')
print(df_period.index[0].start_time, df_period.index[0].end_time)
print(df_period.index[1].start_time, df_period.index[1].end_time)
print(df.index.is_unique, df_period.index.is_unique)
print('*******************************')
df_dt = df_period.to_timestamp()
print(df_dt.head(5))
print(type(df_dt.index))

<class 'pandas.core.indexes.period.PeriodIndex'>
372
         val
date        
2017-12    5
2017-12    6
2018-01    0
2018-01    1
2018-01    2
*******************************
2017-12-01 00:00:00 2017-12-31 23:59:59.999999999
2017-12-01 00:00:00 2017-12-31 23:59:59.999999999
True False
*******************************
            val
date           
2017-12-01    5
2017-12-01    6
2018-01-01    0
2018-01-01    1
2018-01-01    2
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


Pandas 支持使用简单的写法按时间范围筛选数据，如用月份或年份筛选当月或当年的所有数据。

In [130]:
# 筛选和切分
print(df['2019'])  # 筛选2019全年数据
print(df['2019-01'])  #  筛选2019年一月全月数据
print(df['2018':'2019'].head()) # 筛选2018年初到2019年底的所有数据
print(df['2018-12-31':].head()) # 筛选2018-12-31及之后的数据

            val
date           
2019-01-01    1
2019-01-02    2
2019-01-03    3
2019-01-04    4
2019-01-05    5
            val
date           
2019-01-01    1
2019-01-02    2
2019-01-03    3
2019-01-04    4
2019-01-05    5
            val
date           
2018-01-01    0
2018-01-02    1
2018-01-03    2
2018-01-04    3
2018-01-05    4
            val
date           
2018-12-31    0
2019-01-01    1
2019-01-02    2
2019-01-03    3
2019-01-04    4


<ipython-input-130-79d2727d09a2>:2: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  print(df['2019'])  # 筛选2019全年数据
<ipython-input-130-79d2727d09a2>:3: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  print(df['2019-01'])  #  筛选2019年一月全月数据


重采样是指对时序数据按不同的频率重新采样，把高频数据降为低频数据是降采样downsampling，反之则为升采样upsampling。

一种常用的降采样是ohlc方法，它常用于金融领域中计算统计区域内开盘open、最高high、最低low和收盘close的值。

升采样常用于时序数据的插补，如在做时序预测时，缺少某一日期对应的记录，这时就需要升采样补全所有日期范围内的数据。下例中只含有三月份的三条数据，需要补全当月所有日期的数据，按日'D'升采样并使用插值方法插补数据。除了插值方法，常用的插补方法还有使用后面数据插补ffill、使用前面数据插补bfill、使用空值插补fillna等。

In [131]:

# 重采样
tmp = df.resample('w').sum() # 使用叠加方式按周重采样
print(tmp.head(3))

tmp = df.resample('M').ohlc() # 使用ohlc方式按月降采样
print(tmp.head(3))



            val
date           
2017-12-31   11
2018-01-07   21
2018-01-14   21
            val               
           open high low close
date                          
2017-12-31    5    6   5     6
2018-01-31    0    6   0     2
2018-02-28    3    6   0     2


In [149]:
df1 = pd.DataFrame({'val':[8,7,6]})
df1.index = pd.to_datetime(['2019-03-01','2019-03-15','2019-03-31']) # 仅含三条数据
print(df1)
df2 = df1.resample('D').interpolate() # 用插值方式升采样
print(len(df2))
print(df2.head(20))

            val
2019-03-01    8
2019-03-15    7
2019-03-31    6
31
                 val
2019-03-01  8.000000
2019-03-02  7.928571
2019-03-03  7.857143
2019-03-04  7.785714
2019-03-05  7.714286
2019-03-06  7.642857
2019-03-07  7.571429
2019-03-08  7.500000
2019-03-09  7.428571
2019-03-10  7.357143
2019-03-11  7.285714
2019-03-12  7.214286
2019-03-13  7.142857
2019-03-14  7.071429
2019-03-15  7.000000
2019-03-16  6.937500
2019-03-17  6.875000
2019-03-18  6.812500
2019-03-19  6.750000
2019-03-20  6.687500


当时序数据波动较大时，往往会用一段时间的均值来取代该值。在需要计算N天的均值时，可以使用滑动窗口。

DataFrame 的 rolling 方法可以通过对窗口中的数值计算其统计值构造新的字段；emw方法可以实现指数加权滑动窗口，赋予近距离的记录更大的权重。

In [133]:

# 计算滑动窗口
df['sw'] = df['val'].rolling(window=3).mean() # 计算窗口中数据的均值
print(df.head(5))

df['emw_3'] = df['val'].ewm(span=3).mean()
df['emw_7'] = df['val'].ewm(span=7).mean()
df['rolling'] = df['val'].rolling(7).mean()


            val        sw
date                     
2017-12-30    5       NaN
2017-12-31    6       NaN
2018-01-01    0  3.666667
2018-01-02    1  2.333333
2018-01-03    2  1.000000


### 3.4.4  数据重排

数据表转置即行列互换，与矩阵转置类似，使用 DataFrame 自带的方法 T 即可实现。

行转列和列转行
使用DataFrame的stack方法可将原数据表中的列转换为新数据表中的行索引，原行索引不变；unstack方法是将行索引转换成列索引，原列索引不变。stack 方法和 unstack 方法常用于处理多重索引向单层索引的转换

In [136]:
# 数据表转置
df = pd.DataFrame({"a":[1,2],"b":[3,4]}, index=['l1','l2'])
print(df)
print(df.T)
print('*******************************')
# 行转列和列转行
df1 = df.stack() # 列转行
print(df1)
print('*******************************')
print(df1.unstack()) # 将内层行索引转为列索引
print(df1.unstack(level=0)) # 将外层行索引转为列索引

    a  b
l1  1  3
l2  2  4
   l1  l2
a   1   2
b   3   4
*******************************
l1  a    1
    b    3
l2  a    2
    b    4
dtype: int64
*******************************
    a  b
l1  1  3
l2  2  4
   l1  l2
a   1   2
b   3   4


透视转换：
pivot函数提供数据的透视转换功能，它们能将数据的行列按一定规则重组。

In [137]:

# 透视转换
df = pd.DataFrame({"时间":['期中','期末','期中','期末'],
                   "学科":['语文','语文','数学','数学'],
                   "分数":[89,75,90,95]})
print(df)
df1 = df.pivot(index='时间', columns='学科', values='分数')
print('**********************************')
print(df1)

   时间  学科  分数
0  期中  语文  89
1  期末  语文  75
2  期中  数学  90
3  期末  数学  95
**********************************
学科  数学  语文
时间        
期中  90  89
期末  95  75


参考资料：

1.Pandas官方文档https://pandas.pydata.org/docs/

2.http://joyfulpandas.datawhale.club/Home.html

3.《python大数据与机器学习实战》